In [1]:
# Async Connector
# Developed by CD
# v2.0.0-prod

from io import StringIO
import time
import numpy as np
import os
from datetime import datetime, timedelta, date
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy import text
from typing import List
from collections import defaultdict, Counter
import pandas as pd
from cryptography.fernet import Fernet
from dotenv import load_dotenv
from io import StringIO
from pathlib import Path
import asyncio
import nest_asyncio
import sys
from typing import Dict, Union, List
nest_asyncio.apply()

if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())


def retrieve_data(queries: List[Dict[str, Union[str, pd.DataFrame, int]]]) -> Dict[str, pd.DataFrame]:
    """
    Retrieve data from Oracle Database (COCC)

    Args:
        queries (List): pass list of queries in specific format
            - List[Dict[str, Union[str, pd.DataFrame, int]]]
    
    Returns:
        data (Dict): Returns a dictionary with df name and the df attached as key/value pair.   

    """
    class DatabaseHandler:
        """
        This class abstracts the connection to the database and allows a clean
        interface for the developer to use.

        This connector can handle async queries

        """
        def __init__(self, tns_admin_path):
            """
            Args:
                tns_admin_path (str): Oracle driver path
                credentials_path_db1 (str): Database 1 credentials path
                credentials_path_db1 (str): Databsae 2 credentials path
            """
            os.environ['TNS_ADMIN'] = tns_admin_path

            project_root = os.getcwd()
            
            # Load private key
            key_key_path = 'env_admin\key.key'
            with open(key_key_path, "rb") as key_file:
                key = key_file.read()

            cipher = Fernet(key)
            
            # Load encrypted data
            encoded_env_path = r'env_admin\.env.enc'
            with open(encoded_env_path, "rb") as encrypted_file:
                encrypted_data = encrypted_file.read()

            decrypted_data = cipher.decrypt(encrypted_data).decode()

            env_file = StringIO(decrypted_data)
            load_dotenv(stream=env_file)

            self.username1 = os.getenv('main_username')
            self.password1 = os.getenv('main_password')
            self.dsn1 = os.getenv('main_dsn')

            self.username2 = os.getenv('datamart_username')
            self.password2 = os.getenv('datamart_password')
            self.dsn2 = os.getenv('datamart_dsn')

            self.connection_string1 = f'oracle+oracledb://{self.username1}:{self.password1}@{self.dsn1}'
            self.connection_string2 = f'oracle+oracledb://{self.username2}:{self.password2}@{self.dsn2}'

            self.engine1 = create_async_engine(self.connection_string1, max_identifier_length=128, echo=False, future=True)
            self.engine1.dialect.hide_parameters = True
            self.engine2 = create_async_engine(self.connection_string2, max_identifier_length=128, echo=False, future=True)
            self.engine1.dialect.hide_parameters = True


        async def query(self, sql_query, engine=1):
            """
            This allows abstraction of the connection and the class
            so the developer can query a single table as a dataframe

            Args:
                sql_query (str): The query to SQL database is passed as a string
                engine (int): This selects the database. There are two engines:
                    1 -> R1625
                    2 -> COCC DataMart

            Returns:
                df: The SQL query is returned as a pandas DataFrame

            Usage:
                df = db_handler.query("SELECT * FROM DB.TABLE", engine=1)

                In this example, db_handler = DatabaseHandler(args)
            """
            if engine == 1:
                selected_engine = self.engine1
            elif engine == 2:
                selected_engine = self.engine2
            else:
                raise ValueError("Engine must be 1 or 2")

            async with selected_engine.connect() as connection:
                result = await connection.execute(sql_query)
                rows = result.fetchall()
                if not rows:
                    return pd.DataFrame()
                df = pd.DataFrame(rows, columns=result.keys())
            return df

        async def close(self):
            if self.engine1:
                await self.engine1.dispose()
            if self.engine2:
                await self.engine2.dispose()


    # Database Connection Configuration
    tns_admin_path = r'env_admin\tns_admin'
    db_handler = DatabaseHandler(tns_admin_path)

    async def fetch_data(queries):
        try:
            tasks = {query['key']: asyncio.create_task(db_handler.query(query['sql'], query['engine'])) for query in queries}
            results = await asyncio.gather(*tasks.values())
            return {key: df for key, df in zip(tasks.keys(), results)}
        except Exception as e:
            print(f"Error")
            raise
        finally:
            await db_handler.close()

    def run_sql_queries(queries):

        async def run_queries():
            return await fetch_data(queries)
        
        loop = asyncio.get_event_loop()
        if loop.is_running():
            return loop.run_until_complete(run_queries())
        else:
            return asyncio.run(run_queries())
        
    data = run_sql_queries(queries)
    
    return data

In [4]:
# lookup table
# Engine 1
lookup_df = text("""
SELECT 
    *
FROM 
    sys.all_tab_columns col
""")

queries = [
    # {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
    {'key':'lookup_df', 'sql':lookup_df, 'engine':2},
]

data = retrieve_data(queries)
lookup_df = data['lookup_df'].copy()

In [5]:
lookup_df

,owner,table_name,column_name,data_type,data_type_mod,data_type_owner,data_length,data_precision,data_scale,nullable,...,char_used,v80_fmt_image,data_upgraded,histogram,default_on_null,identity_column,evaluation_edition,unusable_before,unusable_beginning,collation
0,XDB,XDB$IMPORT_TT_INFO,ID,RAW,None,None,8,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
1,XDB,XDB$IMPORT_TT_INFO,FLAGS,RAW,None,None,4,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
2,XDB,XDB$IMPORT_TT_INFO,LOCALNAME,VARCHAR2,None,None,2000,NaN,NaN,Y,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP
3,XDB,XDB$IMPORT_TT_INFO,NMSPCID,RAW,None,None,8,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
4,XDB,XDB$IMPORT_TT_INFO,GUID,RAW,None,None,16,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28688,SYS,USER_XML_SCHEMA_ELEMENTS,GLOBAL,RAW,None,None,1,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
28689,SYS,KU$_ASSOC_VIEW,OBJ_TYPE,NUMBER,None,None,22,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
28690,SYS,EXU8VEWU,DEFER,NUMBER,None,None,22,NaN,NaN,Y,...,None,NO,YES,NONE,NO,NO,None,None,None,None
28691,SYS,KU$_USER_VIEW,VERS_MAJOR,CHAR,None,None,1,NaN,NaN,Y,...,B,NO,YES,NONE,NO,NO,None,None,None,USING_NLS_COMP


In [6]:
wh_acctcommon = text("""
SELECT
    a.EFFDATE,
    a.ACCTNBR,
    a.PRODUCT,
    a.ACCTOFFICER,
    a.OWNERSORTNAME,
    a.MJACCTTYPCD,
    a.CURRMIACCTTYPCD,
    a.CURRACCTSTATCD,
    a.NOTEINTRATE,
    a.BOOKBALANCE,
    a.NOTEBAL,
    a.CONTRACTDATE,
    a.CLOSEDATE
FROM
    OSIBANK.WH_ACCTCOMMON a
WHERE
    (a.CURRACCTSTATCD IN ('ACT','DORM')) AND
    (a.MJACCTTYPCD IN ('CK','SAV','TD'))
""")

queries = [
    {'key':'wh_acctcommon', 'sql':wh_acctcommon, 'engine':1},
]

data = retrieve_data(queries)
df = data['wh_acctcommon'].copy()



In [8]:
df[df['acctofficer'] == "JEFFREY P. PAGLIUCA"]

,effdate,acctnbr,product,acctofficer,ownersortname,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,bookbalance,notebal,contractdate,closedate
174,2025-02-26,150621558,Business Elite Money Market,JEFFREY P. PAGLIUCA,CAV INC,CK,CK30,ACT,0.0096,853.26,853.26,2021-06-17,None
1395,2025-02-26,150627902,Business Checking,JEFFREY P. PAGLIUCA,LAND & SEA COLD STORAGE LLC,CK,CK12,ACT,0,3621.37,3621.37,2021-07-06,None
5695,2025-02-26,26152479,Personal Checking,JEFFREY P. PAGLIUCA,"LANG, JEFFREY M.",CK,CK01,ACT,0,2029.84,2029.84,2018-05-17,None
7035,2025-02-26,27078147,Simple Business Checking,JEFFREY P. PAGLIUCA,"D W WHITE CONSTRUCTION, INC.",CK,CK25,ACT,0,13044.15,13044.15,2010-02-05,None
7039,2025-02-26,27078341,Community Checking,JEFFREY P. PAGLIUCA,BUTTONWOOD PARK ZOOLOGICAL SOCIETY INC,CK,CK13,ACT,0.0005,164328.15,164328.15,2010-03-05,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58603,2025-02-26,26174219,Prime Time Checking,JEFFREY P. PAGLIUCA,"SILVA, RAMON D.",CK,CK06,ACT,0.0003,985,985,2017-09-18,None
59177,2025-02-26,151011104,BCSB Elite Savings,JEFFREY P. PAGLIUCA,"SALUTI, DENNIS V.",SAV,SV07,ACT,0.0311,2547.65,2547.65,2024-02-22,None
59698,2025-02-26,27034968,Business Checking,JEFFREY P. PAGLIUCA,WAYLAND DEVELOPMENT CORP,CK,CK12,ACT,0,128635.43,128635.43,2011-04-14,None
61404,2025-02-26,150497313,Simple Business Checking,JEFFREY P. PAGLIUCA,CHRIS' ELECTRONICS CORP,CK,CK25,ACT,0,44633.72,44633.72,2020-08-13,None
